In [ ]:
#| default_exp create_umap_layout

import numpy as np
import umap

# Create umap layout(s)

In [ ]:
#| export

def umap_2d(embeddings: np.ndarray, n_neighbors: int = 15, min_dist: float = 0.1,
                metric: str = "cosine", random_state: int = 42) -> np.ndarray:
    """
    Create a UMAP layout from embeddings.

    Args:
        embeddings: np.ndarray, shape (n_samples, n_features)
        n_neighbors: int, default=15
        min_dist: float, default=0.1
        n_components: int, default=2
    """
    return umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist,
                     metric=metric, random_state=random_state).fit_transform(embeddings)

In [ ]:
#| hide

umap_2d(np.random.rand(5, 10), n_neighbors=15, min_dist=0.1, metric="cosine", random_state=42)

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/umap/umap_.py:2344: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


array([[22.825502, 32.464485],
       [23.419907, 31.209923],
       [23.683466, 32.072388],
       [22.3054  , 31.468697],
       [22.73247 , 30.566425]], dtype=float32)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()